**Autor :           Mati Kukk**  
**Pealkiri:**        **DuckDB "Vaese mehe datalake" lahendus TAI andmete serverless näitamiseks**

In [26]:
# 1. Installid
!pip install duckdb-engine jupysql --quiet

import duckdb
import os
%load_ext sql

# 2. Google Drive mountimine (vajalik ainult Drive'i versiooni jaoks)
# from google.colab import drive
# drive.mount('/content/drive')

# 3. DuckDB ühenduse valikud
# -----------------------------------------------------------
# VARIANT A: DEV (Lokaalne fail Colabi kettal - püsib sessiooni lõpuni) -pool stateless
con = duckdb.connect('/content/tai_avaandmed.db')

# VARIANT B: PROD (Puhas mälusisene - kõige kiirem, aga restardiga tühjeneb) stateless
# con = duckdb.connect()

# VARIANT C: DRIVE (Püsiv andmebaas sinu Google Drive'is)
# Kasuta seda kui tahad et tabelid ja viewd mis oled loonud säoliks
# NB! Enne loo Drive'i kaust 'tai_avaandmed' või muuda teed
# con = duckdb.connect('/content/drive/MyDrive/tai_avaandmed/tai_andmeladu.db')
# -----------------------------------------------------------

# Ühendame SQL magicu
%sql con --alias duckdb
%config SqlMagic.displaylimit = 20

print(f"✅ DuckDB ühendatud")

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
✅ DuckDB ühendatud


In [31]:
# 2. blokk: Metaandmete vaate loomine
USER = "matikukk"
REPO = "tai_avaandmed"

# BASE_URL = f"https://raw.githubusercontent.com/{USER}/{REPO}/main/data" #otselink
BASE_URL = f"https://{USER}.github.io/{REPO}/data" #github pages veidi kiirem


# Loome vaate metaandmetele (ei salvesta andmeid mällu)
con.execute(f"CREATE OR REPLACE VIEW meta AS SELECT * FROM read_parquet('{BASE_URL}/tai_meta.parquet')")

print(f"🌐 Ühendus loodud: {BASE_URL}")
print("✅ Meta-vaade 'meta' on valmis.")

🌐 Ühendus loodud: https://matikukk.github.io/tai_avaandmed/data
✅ Meta-vaade 'meta' on valmis.


In [78]:
# 3. blokk: Andmetabelite vaadete genereerimine
# Loeme tabelite info meta-vaatest
tables = con.execute("SELECT DISTINCT db_schema, table_name FROM meta WHERE obj_type = 't'").fetchall()

print(f"🏗️ Värskendan vaateid {len(tables)} tabelile...")

for schema, table in tables:
    # Loo skeem, kui see on olemas
    if schema:
        con.execute(f"CREATE SCHEMA IF NOT EXISTS {schema}")
        full_name = f"{schema}.{table}"
    else:
        full_name = table

    # Loo vaade otse GitHubi Parquet failile
    file_url = f"{BASE_URL}/{table}.parquet"
    con.execute(f"CREATE OR REPLACE VIEW {full_name} AS SELECT * FROM read_parquet('{file_url}')")

print("✅ Kõik vaated on värskendatud ja SQL magicuks valmis!")

🏗️ Värskendan vaateid 2405 tabelile...
✅ Kõik vaated on värskendatud ja SQL magicuks valmis!


In [ ]:
%%sql
--andmetabelid mis on süsteemist kättesaadavad
select
  db_schema, --schema nimi
  table_name, -- tabeli nimi kuna leidus sama nimega tabeleid siis lisasin ette ka eelmise kihi nime
  ee_description, --kirjeldus eesti keeles
  dt_updated_api, -- allikas uuendamise kuupäev
  src_url, -- allika API url kasutab HTTP POST pole klikitav
  dt_checked -- värskendamise algus
from meta
where obj_type='t';

In [41]:
%config SqlMagic.displaylimit = 20

In [39]:
%%sql
select * from meta where table_name like '%RK01%';

Running query in 'duckdb'

id,db_schema,table_name,src_url,parent_id,obj_type,ee_description,dt_updated_api,dt_created,dt_checked
01Rahvastik.03Abordid.RK01.px,tai_01Rahvastik,tai_03Abordid_RK01_px,https://statistika.tai.ee/api/v1/et/Andmebaas/01Rahvastik/03Abordid/RK01.px,01Rahvastik.03Abordid,t,"RK01: Raseduse katkemised/katkestamised liigi, naise vanuserühma ja elukoha järgi",2025-05-22 13:22:49,2026-02-16 14:15:12.879000,2026-02-18 06:28:45.970384


In [63]:
%%sql
-- puhastame veerunimed
create or replace table stg_abordid_raw as
select
    "ï»¿""Liik""" as liik_raw,
    "Elukoht" as elukoht_raw,
    "VanuserÃ¼hm" as vanus_raw,
    "Aasta" as aasta_raw,
    "Raseduse katkemised ja katkestamised" as kogus_raw
from tai_01rahvastik.tai_03abordid_rk01_px;

Running query in 'duckdb'

Count
30624


In [65]:
%%sql
-- loome staging tabeli
create or replace table stg_abordid (
    liik varchar,
    elukoht varchar,
    vanuseruhm varchar,
    aasta bigint,
    kogus bigint
);

Running query in 'duckdb'

Count


In [72]:
# pythoni funktsiooni encodingu puhastamiseks
# registreerimine duckdb udf-ina

from duckdb.typing import VARCHAR

def fix_encoding(text):
    if text is None:
        return None
    try:
        return text.encode('latin-1').decode('utf-8')
    except:
        return text

# registreerime funktsiooni nimega fn_fix_est
con.create_function("fn_fix_est", fix_encoding, [VARCHAR], VARCHAR)

In [74]:
%%sql
/* stg_abordid täitmine: udf kasutamine puhastamiseks */
-- kasutame registreeritud funktsiooni fn_fix_est kõigil tekstiväljadel
-- dbt stiilis väikesed tähed ja selge insert-loogika
truncate table stg_abordid;

insert into stg_abordid
select
    fn_fix_est(trim(liik_raw)) as liik,
    fn_fix_est(trim(elukoht_raw)) as elukoht,
    fn_fix_est(trim(vanus_raw)) as vanuseruhm,
    aasta_raw as aasta,
    kogus_raw as kogus
from stg_abordid_raw;

Running query in 'duckdb'

Count
30624


In [ ]:
%%sql
--kontroll
from stg_abordid;

In [76]:
%%sql
/* f_abordid: koondridade eemaldamine ja lõplik filtreerimine */
-- eemaldame "kokku" read, et analüüsi käigus andmeid mitte dubleerida

create or replace table f_abordid as
select
    liik,
    elukoht,
    vanuseruhm,
    aasta,
    kogus
from stg_abordid
where elukoht not ilike '%kokku%'
  and vanuseruhm not ilike '%kokku%'
  and liik not ilike '%kokku%';

Running query in 'duckdb'

Count
28072


In [71]:
# visualiseering


import plotly.express as px
import pandas as pd

# 1. laeme andmed duckdb-st pandas dataframe-i
# grupeerime andmed aasta ja liigi kaupa, et trendid välja joonistuksid
df = con.sql("""
    select
        aasta,
        liik,
        sum(kogus) as kogus
    from f_abordid
    group by 1, 2
    order by aasta, kogus desc
""").df()

# 2. loome interaktiivse joondiagrammi
# kasutame 'liik' muutujat värvide eristamiseks
fig = px.line(
    df,
    x="aasta",
    y="kogus",
    color="liik",
    title="Abortide arv Eestis aastate ja liikide lõikes",
    labels={"aasta": "Aasta", "kogus": "Juhtude arv", "liik": "Abordi liik"},
    template="plotly_white"
)

# 3. kuvame graafiku
fig.show()

In [77]:
%config SqlMagic.displaylimit = 12